# Riddler Classic 2022-04-08

https://fivethirtyeight.com/features/can-you-be-mediocre-enough-to-win/

*This week’s Classic also comes from Ben Orlin, although it’s relatively mediocre compared to the Express.*

*In the three-player Game of Mediocrity, you win by not winning too much.*

*Each round, every player secretly picks a number from 0 to 10. The numbers are simultaneously revealed, and the median number wins that number of points. (If two or more players pick the same number, then the winner is randomly selected from among them.)*

*Player A says 6, B says 3, and C says 2. B is the winner, having said the median, and earns 3 points.
After five rounds, the winner is whoever has the median number of points. (Again, if two or more players have the same score, then the winner is randomly selected from among them.)*

*With one round remaining, players A, B and C have 6, 8 and 10 points, respectively. Player A sighs and writes down “3,” but fails to do so in secret. Players B and C both see player A’s number (and both see that the other saw A’s number), and will take care to write their own numbers in secret. Assuming everyone plays to win, what numbers should B and C choose?*

Let's start by calculating conditional probabilities of winning for all of each of the two relevant player's choices.

In [2]:
import statistics
import pandas as pd

In [15]:
#Conditional probabilities for player B's options

a_score = 6
b_score = 8
c_score = 10

for b in range(0, 11):
    success = 0
    for c in range(0, 11):
        round_median = statistics.median([3, b, c])
        round_winners = [3, b, c].count(round_median)
        a_wins = (3 == round_median)
        b_wins = (b == round_median)
        c_wins = (c == round_median)
        if a_wins:
            score_median = statistics.median([a_score + round_median, b_score, c_score])
            game_winners = [a_score + round_median, b_score, c_score].count(score_median)
            success += int(b_score == score_median) / round_winners / game_winners
        if b_wins:
            score_median = statistics.median([a_score, b_score + round_median, c_score])
            game_winners = [a_score, b_score + round_median, c_score].count(score_median)
            success += int(b_score == score_median) / round_winners / game_winners
        if c_wins:
            score_median = statistics.median([a_score, b_score, c_score + round_median])
            game_winners = [a_score, b_score, c_score + round_median].count(score_median)
            success += int(b_score == score_median) / round_winners / game_winners
            
    print("P(B wins | B picks " + str(b) + ") =",  str(round(success/11, 2)))

P(B wins | B picks 0) = 0.32
P(B wins | B picks 1) = 0.18
P(B wins | B picks 2) = 0.09
P(B wins | B picks 3) = 0.03
P(B wins | B picks 4) = 0.09
P(B wins | B picks 5) = 0.18
P(B wins | B picks 6) = 0.27
P(B wins | B picks 7) = 0.36
P(B wins | B picks 8) = 0.45
P(B wins | B picks 9) = 0.55
P(B wins | B picks 10) = 0.64


In [16]:
#Conditional probabilities for player C's options
for c in range(0, 11):
    success = 0
    for b in range(0, 11):
        round_median = statistics.median([3, b, c])
        round_winners = [3, b, c].count(round_median)
        a_wins = (3 == round_median)
        b_wins = (b == round_median)
        c_wins = (c == round_median)
        if a_wins:
            score_median = statistics.median([a_score + round_median, b_score, c_score])
            game_winners = [a_score + round_median, b_score, c_score].count(score_median)
            success += int(c_score == score_median) / round_winners / game_winners
        if b_wins:
            score_median = statistics.median([a_score, b_score + round_median, c_score])
            game_winners = [a_score, b_score + round_median, c_score].count(score_median)
            success += int(c_score == score_median) / round_winners / game_winners
        if c_wins:
            score_median = statistics.median([a_score, b_score, c_score + round_median])
            game_winners = [a_score, b_score, c_score + round_median].count(score_median)
            success += int(c_score == score_median) / round_winners / game_winners
    print("P(C wins | C picks " + str(c) + ") =", str(round(success/11, 2)))


P(C wins | C picks 0) = 0.09
P(C wins | C picks 1) = 0.09
P(C wins | C picks 2) = 0.07
P(C wins | C picks 3) = 0.03
P(C wins | C picks 4) = 0.09
P(C wins | C picks 5) = 0.18
P(C wins | C picks 6) = 0.27
P(C wins | C picks 7) = 0.36
P(C wins | C picks 8) = 0.45
P(C wins | C picks 9) = 0.55
P(C wins | C picks 10) = 0.64


If players B and C knew that the other player hadn't seen player A's final number, they would each pick 10 to optimize their chances of winning. Instead, this becomes a game theory problem, and we need to find the strict Nash equilibrium. The strict Nash equilibrium will be the outcome which gives players B and C the highest chance of winning assuming the other player acts rationally. 

What exactly would rational behavior look like? Well, for starters, players B and C can eliminate any chance of player A winning by ensuring the median is a number other than 3. This means players B and C will not pick option 3. Other than this, finding probabilities becomes complicated, but we know each player will want to consider all of the other player's possible moves. Let's continue by writing some more code.

In [22]:
df = pd.DataFrame()
for c in range(0, 11):
    P_max = (0, 0)
    for b in range(0, 11):
        P = 0
        round_median = statistics.median([3, b, c])
        round_winners = [3, b, c].count(round_median)
        a_wins = (3 == round_median)
        b_wins = (b == round_median)
        c_wins = (c == round_median)
        if a_wins:
            score_median = statistics.median([a_score + round_median, b_score, c_score])
            game_winners = [a_score + round_median, b_score, c_score].count(score_median)
            P += int(b_score == score_median) / round_winners / game_winners
        if b_wins:
            score_median = statistics.median([a_score, b_score + round_median, c_score])
            game_winners = [a_score, b_score + round_median, c_score].count(score_median)
            P += int(b_score == score_median) / round_winners / game_winners
        if c_wins:
            score_median = statistics.median([a_score, b_score, c_score + round_median])
            game_winners = [a_score, b_score, c_score + round_median].count(score_median)
            P += int(b_score == score_median) / round_winners / game_winners
        if P >= P_max[1]:
            P_max = (b, P)
    df = df.append(pd.DataFrame({"Player C's Pick": [c], "Player B's Optimal Pick": [P_max[0]], "P(B Wins | B Picks Optimally)": [round(P_max[1], 2)]}))

print(df.to_string(index = False))

print()

df = pd.DataFrame()
for b in range(0, 11):
    P_max = (0, 0)
    for c in range(0, 11):
        P = 0
        round_median = statistics.median([3, b, c])
        round_winners = [3, b, c].count(round_median)
        a_wins = (3 == round_median)
        b_wins = (b == round_median)
        c_wins = (c == round_median)
        if a_wins:
            score_median = statistics.median([a_score + round_median, b_score, c_score])
            game_winners = [a_score + round_median, b_score, c_score].count(score_median)
            P += int(c_score == score_median) / round_winners / game_winners
        if b_wins:
            score_median = statistics.median([a_score, b_score + round_median, c_score])
            game_winners = [a_score, b_score + round_median, c_score].count(score_median)
            P += int(c_score == score_median) / round_winners / game_winners
        if c_wins:
            score_median = statistics.median([a_score, b_score, c_score + round_median])
            game_winners = [a_score, b_score, c_score + round_median].count(score_median)
            P += int(c_score == score_median) / round_winners / game_winners
        if P >= P_max[1]:
            P_max = (c, P)
    df = df.append(pd.DataFrame({"Player B's Pick": [b], "Player C's Optimal Pick": [P_max[0]], "P(C Wins | C Picks Optimally)": [round(P_max[1], 2)]}))

print(df.to_string(index = False))

 Player C's Pick  Player B's Optimal Pick  P(B Wins | B Picks Optimally)
               0                        0                            1.0
               1                        0                            1.0
               2                        1                            1.0
               3                       10                            0.5
               4                       10                            1.0
               5                       10                            1.0
               6                       10                            1.0
               7                       10                            1.0
               8                       10                            1.0
               9                       10                            1.0
              10                       10                            0.5

 Player B's Pick  Player C's Optimal Pick  P(C Wins | C Picks Optimally)
               0                       10         

As shown by the above data frames, the strict Nash equilibrium involves players B and C both writing down "10". Player B has optimal choices other than 10 if player C picks a number less than 3. Player C needs the median to be greater than 3 to win, however, so this would never happen. Thus, player B will pick 10. It follows that player C will pick 10 from the second data frame. Players B and C will each have a 50% chance of winning.